# Projeto de Classificação para Marinha do Brasil

## Autor: Natanael Junior (natmourajr@gmail.com)

Laboratório de Processamento de Sinais - UFRJ

Laboratório de Tecnologia Sonar

Instituto de Pesquisas da Marinha - IPqM

# Bibliotecas e leitura dos dados
As bibliotecas necessárias para a inclusão

In [ ]:
import os
import pickle
import numpy as np
import time

init_time = time.time()

from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.externals import joblib

m_time = time.time()
print 'Time to import all libraries: %1.5f seconds'%(m_time-init_time)

outputpath = os.environ['OUTPUTDATAPATH']
main_analysis_path = os.environ['SONAR_WORKSPACE']
log_analysis_path = os.environ['PACKAGE_OUTPUT']
result_analysis_path = os.environ['PACKAGE_OUTPUT']+'/PCASingleClassSVM'
# Read data
# Check if LofarData has created...
m_time = time.time()


subfolder = '4classes_old'
n_pts_fft = 1024
decimation_rate = 3

if(not os.path.exists(outputpath+'/'+'LofarData_%s_%i_fft_pts_%i_decimation_rate.jbl'%(
            subfolder,n_pts_fft,decimation_rate))):
    print outputpath+'/'+'LofarData_%s_%i_fft_pts_%i_decimation_rate.jbl'%(
        subfolder,n_pts_fft,decimation_rate)+' doesnt exist...please create it'
    exit()
    
#Read lofar data
[data,class_labels] = joblib.load(outputpath+'/'+'LofarData_%s_%i_fft_pts_%i_decimation_rate.jbl'%(
            subfolder,n_pts_fft,decimation_rate))

m_time = time.time()-m_time
print 'Time to read data file: %1.5f seconds'%m_time

Time to import all libraries: 4.73550 seconds
/Users/natmourajr/Workspace/Doutorado/SonarAnalysis/Results/LofarData_4classes_old_1024_fft_pts_3_decimation_rate.jbl doesnt exist...please create it


IOError: [Errno 2] No such file or directory: '/Users/natmourajr/Workspace/Doutorado/SonarAnalysis/Results/LofarData_4classes_old_1024_fft_pts_3_decimation_rate.jbl'

# Processamento dos dados
Os dados encontram-se no formato do matlab, para isso precisam ser processados para o formato de python.

In [ ]:
# Process data...
# create a full data vector
all_data = {};
all_trgt = {};

for iclass, class_label in enumerate(class_labels):
    for irun in range(len(data[iclass])):
        if len(all_data) == 0:
            all_data = data[iclass][irun]['Signal']
            all_trgt = (iclass)*np.ones(data[iclass][irun]['Signal'].shape[1])
        else:
            all_data = np.append(all_data,data[iclass][irun]['Signal'],axis=1)
            all_trgt = np.append(all_trgt,(iclass)*np.ones(data[iclass][irun]
                                                           ['Signal'].shape[1]),axis=0)
            
all_data = all_data.transpose()

# Balanceamento de Classes
Os dados encontram-se desbalanceados. Com isso, os classificadores podem se especializar em uma classe (gerando mais SVs para a mesma) e não se especializar em outras

Acessados em 21/12/2016

https://svds.com/learning-imbalanced-classes/

http://www.cs.utah.edu/~piyush/teaching/ImbalancedLearning.pdf

http://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html

Para solucionar isso, a primeira solução é "criar" dados das classes com menos eventos de maneira aleatória. Outras soluções podem ser propostas posteriormente.

In [ ]:
# Process data
# unbalanced data to balanced data with random data creation of small classes

# Same number of events in each class
qtd_events_biggest_class = 0
biggest_class_label = ''

for iclass, class_label in enumerate(class_labels):
    if sum(all_trgt==iclass) > qtd_events_biggest_class:
        qtd_events_biggest_class = sum(all_trgt==iclass)
        biggest_class_label = class_label
    print "Qtd event of %s is %i"%(class_label,sum(all_trgt==iclass))
print "\nBiggest class is %s with %i events"%(biggest_class_label,qtd_events_biggest_class)


balanced_data = {}
balanced_trgt = {}

from Functions import DataHandler as dh
m_datahandler = dh.DataHandlerFunctions()

for iclass, class_label in enumerate(class_labels):
    if len(balanced_data) == 0:
        class_events = all_data[all_trgt==iclass,:]
        balanced_data = m_datahandler.CreateEventsForClass(
            class_events,qtd_events_biggest_class-(len(class_events)))
        balanced_trgt = (iclass)*np.ones(qtd_events_biggest_class)
    else:
        balanced_data = np.append(balanced_data,
                                  (m_datahandler.CreateEventsForClass(
                    all_data[all_trgt==iclass,:],
                    qtd_events_biggest_class-sum(all_trgt==iclass))),
                                  axis=0)
        balanced_trgt = np.append(balanced_trgt,
                                  (iclass)*np.ones(qtd_events_biggest_class),axis=0)
        
all_data = balanced_data
all_trgt = balanced_trgt

# Definições do treinamento
Nessa célula temos os parâmetros do treinamento a ser realizado. No log, deve ficar armazenada a data do treinamento para a reconstrução dos resultados

In [ ]:
%time

from Functions import LogFunctions as log

# Create a entry in log file
m_log = log.LogInformation()
date = m_log.CreateLogEntry("NoveltyDetection",'PCASingleClassSVM')

# Create a train information file
n_folds = 2
n_pcas = 270
norm = 'mapstd'
#nu_values = np.array([0.7, 0.8, 0.9])
#nu_values = np.array([0.001, 0.1, 0.2])
nu_values = np.array([0.001, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.999])
gamma_value = 0.0001

train_info = {}
train_info['n_folds'] = n_folds
train_info['n_pcas'] = n_pcas
train_info['norm'] = norm
train_info['nu_values'] = nu_values
train_info['gamma_value'] = gamma_value

# divide data in train and test for novelty detection
for novelty_class, novelty_label in enumerate(class_labels):
    print 'Dividing data in trn and tst for novelty class: %s'%(novelty_label)
    CVO = cross_validation.StratifiedKFold(all_trgt[all_trgt!=novelty_class], n_folds)
    CVO = list(CVO)
    train_info['CVO_novelty_%s'%(novelty_label)] = CVO

train_info['preprocessing_done'] = False
train_info['preprocessing_analysis'] = False
train_info['train_done'] = False
train_info['results_done'] = False

train_info_name = result_analysis_path+'/train_info_files'+'/'+date+'_train_info.jbl'
joblib.dump([train_info],train_info_name,compress=9)


In [ ]:
# Read log files
from Functions import LogFunctions as log
mlog = log.LogInformation()
log_entries = mlog.RecoverLogEntries(package_name="NoveltyDetection")
print log_entries

In [ ]:
# Read Information of Train Info File
choose_date = '2017_05_09_00_09_35'

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'Analysing train performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    print 'PCASingleClassSVM Train Info File'
    print 'Date: %s'%(choose_date)
    print 'Number of Folds: %i'%(train_info['n_folds'])
    print 'Number of Used PCAs: %i'%(train_info['n_pcas'])
    print 'Normalization Method: %s'%(train_info['norm'])
    print 'Gamma Value: %1.7f'%(train_info['gamma_value'])
    print 'Nu Value(s): '
    print train_info['nu_values']
    if train_info['preprocessing_done']:
        print 'Preprocessing Done: True'
    else:
        print 'Preprocessing Done: False'
    if train_info['preprocessing_analysis']:
        print 'Preprocessing Analysis Done: True'
    else:
        print 'Preprocessing Analysis Done: False'
    
    if train_info['train_done']:
        print 'Train Done: True'
    else:
        print 'Train Done: False'
    if train_info['results_done']:
        print 'Extract Results: True'
    else:
        print 'Extract Results: False'

# Preprocessamento - PCA
Como a dimensionalidade dos dados é alta (400 dimensões), um pré-processamento se faz necessário para reduzir as dimensões das entradas e tornar o modelo menos complexo. Aqui, o pré-processamento utilizado é a PCA (análise de componentes principais) linear.

## Arquivos gerados
Uma vez que uma das classes seja removida do processo de treinamento (considerada novidade), podemos fazer a extração dos PCAs para as classes restantes. Para isso, devemos definir um conjunto de treinamento e de testes. No conjunto de treinamento, os parâmetros (PCAs) serão extraídos, enquanto no conjunto de teste, o poder de generalização do processo de extração será testado.

In [ ]:
# PCA extraction
%time

pcas = {}

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'PCA extraction performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    
    # saving time
    if train_info['preprocessing_done']:
        print 'Preprocessing done, just analyse it'
        continue

    for novelty_class, novelty_label in enumerate(class_labels):
        pcas[novelty_class] = {}
        print 'Extracting PCA for novelty %s'%(novelty_label)
        
        known_data = all_data[all_trgt!=novelty_class,:]
        
        for ifold in range(len(train_info['CVO_novelty_%s'%(novelty_label)])):
            # split data in trn set, tst set
            train_id, test_id = train_info['CVO_novelty_%s'%(novelty_label)][ifold]

            # normalize data based in train set
            if train_info['norm'] == 'mapstd':
                scaler = preprocessing.StandardScaler().fit(known_data[train_id,:])
            elif train_info['norm'] == 'mapstd_rob':
                scaler = preprocessing.RobustScaler().fit(known_data[train_id,:])
            elif train_info['norm'] == 'mapminmax':
                scaler = preprocessing.MinMaxScaler().fit(known_data[train_id,:])
                
            norm_known_data = scaler.transform(known_data)
            
            pca = PCA(n_components=known_data.shape[1])
            
            pca.fit(norm_known_data[train_id,:])
            pcas[novelty_class][ifold] = pca
        
    # saving file
    pca_file_path = result_analysis_path+'/result_files'+'/'+choose_date+'_pca_file.jbl'

    if pcas != {}:
        joblib.dump([pcas],pca_file_path,compress=9)

    train_info_name = result_analysis_path+'/train_info_files'+'/'+choose_date+'_train_info.jbl'

    train_info['preprocessing_done'] = True
    joblib.dump([train_info],train_info_name,compress=9)

In [ ]:
# PCA energy analysis - energy curve
%time

pcas_energy = {}
novelty_error = {}

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'PCA analysis performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    
    # saving time
    if not train_info['preprocessing_done']:
        print 'Preprocessing is not done, just do it'
        continue
    
    [pcas] = joblib.load(result_analysis_path+'/result_files'+'/'+choose_date+'_pca_file.jbl')
    
    for novelty_class, novelty_label in enumerate(class_labels): 
        print 'Novelty: %s'%(novelty_label)
        pcas_energy[novelty_class] = np.zeros([train_info['n_folds'],all_data.shape[1]])
        novelty_error[novelty_class] = np.zeros([train_info['n_folds'],all_data.shape[1]])
        
        novelty_data = all_data[all_trgt==novelty_class,:]
        known_data = all_data[all_trgt!=novelty_class,:]
        
        for ifold in range(len(train_info['CVO_novelty_%s'%(novelty_label)])):
            pcas_energy[novelty_class][ifold,:] = (pcas[novelty_class][ifold].explained_variance_/
                                                   np.sum(pcas[novelty_class][ifold].explained_variance_))
            
            # split data in trn set, tst set
            train_id, test_id = train_info['CVO_novelty_%s'%(novelty_label)][ifold]

            # normalize data based in train set
            if train_info['norm'] == 'mapstd':
                scaler = preprocessing.StandardScaler().fit(known_data[train_id,:])
            elif train_info['norm'] == 'mapstd_rob':
                scaler = preprocessing.RobustScaler().fit(known_data[train_id,:])
            elif train_info['norm'] == 'mapminmax':
                scaler = preprocessing.MinMaxScaler().fit(known_data[train_id,:])
                
            norm_known_data = scaler.transform(known_data)
            
            for ipca in range(all_data.shape[1]):
                pca = PCA(n_components=ipca)
                pca.fit(norm_known_data[train_id,:])
                tran_novelty_data = pca.transform(novelty_data)
                reco_novelty_data = pca.inverse_transform(tran_novelty_data)
                novelty_error[novelty_class][ifold,ipca] = np.mean((novelty_data-reco_novelty_data)**2)
                
            
    
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['xtick.labelsize'] = 13
    plt.rcParams['ytick.labelsize'] = 13
    
    n_pcas = all_data.shape[1]
    bins = np.linspace(0,n_pcas-1,n_pcas)
    
    n_columns = 1
    icolumn = 0
    
    n_rows = 4
    irow = 0
    
    fig, ax = plt.subplots(nrows=n_rows, ncols=n_columns,figsize=(10,20))
    
    
    for novelty_class, novelty_label in enumerate(class_labels):
        #ax[irow,icolumn].plot(bins+0.5,100.*np.cumsum(np.mean(pcas_energy[novelty_class].T,1)),'r--')
        ax[irow].errorbar(bins+0.5,
                                 100.*np.cumsum(np.mean(pcas_energy[novelty_class].T,1)),
                                 np.std(pcas_energy[novelty_class].T,1),fmt='-o')
        m_leg = []
        
        m_leg.append('PCA Energy')
        
        ax[irow].errorbar(bins+0.5,
                                 1000.*np.mean(novelty_error[novelty_class].T,axis=1),
                                 np.std(novelty_error[novelty_class].T,axis=1),fmt='-o')
        m_leg.append('Novelty Error (MSE)')
        ax[irow].set_ylim([0,100])
        
        
        m_fontsize = 13
        ax[irow].set_title('PCA Extraction for %s novelty'%(novelty_label), 
                                   fontsize= m_fontsize, fontweight='bold')
        ax[irow].set_ylabel('% of Cum. Energy/100.0*MSE', fontsize= m_fontsize, fontweight='bold')
        if novelty_class == 3:
            ax[irow].set_xlabel('Number of Components', fontsize= m_fontsize, fontweight='bold')
        ax[irow].grid()
        
        ax[irow].set_yticks([x for x in range(0,101,5)])
        ax[irow].set_xticks([x for x in range(0,401,20)])
        ax[irow].set_xticklabels([x for x in range(0,401,20)],rotation=45)
        ax[irow].legend(m_leg, loc='upper left',ncol=1,fontsize=m_fontsize-3)
        
        for ipca in range(0,n_pcas,1):
            if np.sum(np.mean(pcas_energy[novelty_class].T,1)[0:ipca]) > 0.9:
                break
        ax[irow].plot(bins[ipca],100.*np.sum(np.mean(pcas_energy[novelty_class].T,1)[0:ipca]),
                              'ro',markersize=16)
        
                
        if icolumn+1 == n_columns:
            icolumn = 0
            irow = irow+1
        else:
            icolumn = icolumn+1
    plt.tight_layout()
    fig.savefig(result_analysis_path+'/picts/'+choose_date+'_pca_cumulative_energy.pdf')

In [ ]:
result_analysis_path+'/picts/'+choose_date+'_pca_cumulative_energy.pdf'

In [ ]:
# PCA energy analysis - amount of energy per pca
%time

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'PCA analysis performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    
    # saving time
    if not train_info['preprocessing_done']:
        print 'Preprocessing is not done, just do it'
        continue
    
    
    [pcas] = joblib.load(result_analysis_path+'/result_files'+'/'+choose_date+'_pca_file.jbl')
    
    for novelty_class, novelty_label in enumerate(class_labels): 
        pcas_energy[novelty_class] = np.zeros([train_info['n_folds'],all_data.shape[1]])
        
        for ifold in range(len(train_info['CVO_novelty_%s'%(novelty_label)])):
            pcas_energy[novelty_class][ifold,:] = (pcas[novelty_class][ifold].explained_variance_/
                                                   np.sum(pcas[novelty_class][ifold].explained_variance_))
            
    import matplotlib.pyplot as plt
    %matplotlib inline
    
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['xtick.labelsize'] = 18
    plt.rcParams['ytick.labelsize'] = 18
    
    n_pcas = all_data.shape[1]
    bins = np.linspace(0,n_pcas-1,n_pcas)
    
    n_columns = 2
    icolumn = 0
    
    n_rows = 2
    irow = 0
    
    fig, ax = plt.subplots(nrows=n_rows, ncols=n_columns,figsize=(25,20))
    
    
    for novelty_class, novelty_label in enumerate(class_labels):
        
        
        plot_data = np.mean(pcas_energy[novelty_class].T,1)
        err_plot_data = np.var(pcas_energy[novelty_class].T,1)
        ax[irow,icolumn].bar(bins,plot_data,yerr=err_plot_data, width=1,facecolor='b',alpha=0.75)
        ax[irow,icolumn].errorbar(bins+0.5, plot_data,yerr=err_plot_data,ecolor='r',fmt='o')
        
        #ax[irow,icolumn].set_ylim([0,100])
        
        m_fontsize = 25
        ax[irow,icolumn].set_title('PCA Extraction for %s novelty'%(novelty_label), 
                                   fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].set_ylabel('% of Energy', fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].set_xlabel('#PCA', fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].grid()
        
        #ax[irow,icolumn].set_yticks([x for x in range(0,101,5)])
        ax[irow,icolumn].set_xticks([x for x in range(0,401,20)])
        ax[irow,icolumn].set_xticklabels([x for x in range(0,401,20)],rotation=45)
                
        
        if icolumn+1 == n_columns:
            icolumn = 0
            irow = irow+1
        else:
            icolumn = icolumn+1
    
    fig.savefig(result_analysis_path+'/picts/'+choose_date+'_pca_amount_energy.pdf')

In [ ]:
# PCA energy analysis - correlation before PCA
%time

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'PCA analysis performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    
            
    import matplotlib.pyplot as plt
    %matplotlib inline
    
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['xtick.labelsize'] = 18
    plt.rcParams['ytick.labelsize'] = 18
    
    n_pcas = all_data.shape[1]
    bins = np.linspace(0,n_pcas-1,n_pcas)
    
    n_columns = 2
    icolumn = 0
    
    n_rows = 2
    irow = 0
    
    fig, ax = plt.subplots(nrows=n_rows, ncols=n_columns,figsize=(25,20))
    
    
    for novelty_class, novelty_label in enumerate(class_labels):
        known_data = all_data[all_trgt!=novelty_class,:]
        known_trgt = all_trgt[all_trgt!=novelty_class]
        
        corr_mat = np.corrcoef(known_data[:,:],rowvar=False)
        
        im = ax[irow,icolumn].imshow(corr_mat, interpolation='nearest',clim=(-1.0, 1.0))
        plt.colorbar(im,ax=ax[irow,icolumn])
        
        m_fontsize = 25
        ax[irow,icolumn].set_title('Correlation Coeficient for %s novelty'%(novelty_label), 
                                   fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].set_ylabel('# Frequency bin', fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].set_xlabel('# Frequency bin', fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].grid()
        
        ax[irow,icolumn].set_yticks([x for x in range(0,401,20)])
        ax[irow,icolumn].set_yticklabels([x for x in range(0,401,20)],rotation=45)
        ax[irow,icolumn].set_xticks([x for x in range(0,401,20)])
        ax[irow,icolumn].set_xticklabels([x for x in range(0,401,20)],rotation=45)
        
        
        if icolumn+1 == n_columns:
            icolumn = 0
            irow = irow+1
        else:
            icolumn = icolumn+1
    
    fig.savefig(result_analysis_path+'/picts/'+choose_date+'_before_pca_corr_coef.pdf')

In [ ]:
# PCA energy analysis - correlation after PCA
%time

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'PCA analysis performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    
    # saving time
    if not train_info['preprocessing_done']:
        print 'Preprocessing is not done, just do it'
        continue
    
    
    [pcas] = joblib.load(result_analysis_path+'/result_files'+'/'+choose_date+'_pca_file.jbl')
    
    choose_fold = 0
    
    import matplotlib.pyplot as plt
    %matplotlib inline
    
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['xtick.labelsize'] = 18
    plt.rcParams['ytick.labelsize'] = 18
    
    n_pcas = all_data.shape[1]
    bins = np.linspace(0,n_pcas-1,n_pcas)
    
    n_columns = 2
    icolumn = 0
    
    n_rows = 2
    irow = 0
    
    fig, ax = plt.subplots(nrows=n_rows, ncols=n_columns,figsize=(25,20))
    
    
    for novelty_class, novelty_label in enumerate(class_labels):
        
        my_pca = pcas[novelty_class][choose_fold]
        
        known_data = all_data[all_trgt!=novelty_class,:]
        known_trgt = all_trgt[all_trgt!=novelty_class]
        
        pca_proj_data = my_pca.transform(known_data)
        
        corr_mat = np.corrcoef(pca_proj_data[:,:],rowvar=False)
        
        im = ax[irow,icolumn].imshow(corr_mat, interpolation='nearest',clim=(-1.0, 1.0))
        plt.colorbar(im,ax=ax[irow,icolumn])
        
        m_fontsize = 25
        ax[irow,icolumn].set_title('Correlation Coeficient for %s novelty'%(novelty_label), 
                                   fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].set_ylabel('# Frequency bin', fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].set_xlabel('# Frequency bin', fontsize= m_fontsize, fontweight='bold')
        ax[irow,icolumn].grid()
        
        ax[irow,icolumn].set_yticks([x for x in range(0,401,20)])
        ax[irow,icolumn].set_yticklabels([x for x in range(0,401,20)],rotation=45)
        ax[irow,icolumn].set_xticks([x for x in range(0,401,20)])
        ax[irow,icolumn].set_xticklabels([x for x in range(0,401,20)],rotation=45)
        
        
        if icolumn+1 == n_columns:
            icolumn = 0
            irow = irow+1
        else:
            icolumn = icolumn+1
    
    fig.savefig(result_analysis_path+'/picts/'+choose_date+'_after_pca_corr_coef.pdf')

# Treinamento - Single Class SVM
Adicionar informações
## Arquivos gerados
Adicionar informações

In [ ]:
# Treinamento do detector de novidade

%time

# Classifiers
classifiers = {}


for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'PCA analysis performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    
    
    # saving time
    if not train_info['preprocessing_done']:
        print 'Pre-processing is not done'
        continue
    
    # saving time
    if train_info['train_done']:
        print 'Train done, just analyse it'
        continue
    
    [pcas] = joblib.load(result_analysis_path+'/result_files'+'/'+choose_date+'_pca_file.jbl')
    
    for novelty_class, novelty_label in enumerate(class_labels):
        classifiers[novelty_class] = {}
        
        known_data = all_data[all_trgt!=novelty_class,:]
        known_trgt = all_trgt[all_trgt!=novelty_class]
        
        # for: folds
        for ifold in range(len(train_info['CVO_novelty_%s'%(novelty_label)])):
            classifiers[novelty_class][ifold] = {}
            
            # split data in trn set, tst set
            train_id, test_id = train_info['CVO_novelty_%s'%(novelty_label)][ifold]
            
            # normalize data based in train set
            if train_info['norm'] == 'mapstd':
                scaler = preprocessing.StandardScaler().fit(known_data[train_id,:])
            elif train_info['norm'] == 'mapstd_rob':
                scaler = preprocessing.RobustScaler().fit(known_data[train_id,:])
            elif train_info['norm'] == 'mapminmax':
                scaler = preprocessing.MinMaxScaler().fit(known_data[train_id,:])
        
            norm_known_data = scaler.transform(known_data)
        
            pca_norm_known_data = pcas[novelty_class][ifold].transform(norm_known_data)
            
            #cut some pcas
            pca_norm_known_data = pca_norm_known_data[:,0:train_info['n_pcas']]
        
            for nu_id, nu_value in enumerate(train_info['nu_values']):
                classifiers[novelty_class][ifold][nu_value] = {}
                print 'Training Classifiers: Novelty Class: %s, %i of %i folds, %i of %i nu_values'%(
                    novelty_label, ifold+1, len(train_info['CVO_novelty_%s'%(novelty_label)]),
                    nu_id+1, len(train_info['nu_values'])
                )
                
                # novelty detector
                novelty_detector = svm.OneClassSVM(nu=nu_value, 
                                                   kernel="rbf", 
                                                   gamma=train_info['gamma_value'])
                novelty_detector.fit(pca_norm_known_data[train_id,:])
                
                classifiers[novelty_class][ifold][nu_value]['NoveltyDetector'] = novelty_detector
                
                # class specialist
                for known_class, known_label in enumerate(class_labels):
                    if known_class == novelty_class: continue
                    class_idx = np.nonzero(known_trgt == known_class)[0]
                    idx = np.intersect1d(class_idx,train_id)
                    classifier = svm.OneClassSVM(nu=nu_value, kernel="rbf", 
                                                 gamma=train_info['gamma_value'])
                    classifier.fit(pca_norm_known_data[idx,:])
                    classifiers[novelty_class][ifold][nu_value][known_label] = classifier
    
        classifier_info_name = result_analysis_path+'/classifiers_files'+'/'+choose_date+'_'+novelty_label+'_classifiers.jbl'
        joblib.dump([classifiers[novelty_class]],classifier_info_name,compress=9)

train_info_name = result_analysis_path+'/train_info_files'+'/'+choose_date+'_train_info.jbl'

train_info['train_done'] = True
joblib.dump([train_info],train_info_name,compress=9)

In [ ]:
# Extração dos resultados
%time

import scipy.stats as sp_stats

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'PCA analysis performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    
    # checking preprocessing
    if not train_info['preprocessing_done']:
        print 'Extract Preprocessing!!!'
        continue
        
    # checking training
    #if not train_info['train_done']:
    #    print 'Perform Train!!!'
    #    continue
        
    # checking results extraction
    if train_info['results_done']:
        print 'Extraction done, just analyse it'
        continue
        
    [pcas] = joblib.load(result_analysis_path+'/result_files'+'/'+choose_date+'_pca_file.jbl')
    
    #classifier_info_name = result_analysis_path+'/classifiers_files'+'/'+choose_date+'_classifiers.jbl'
    #[classifiers] = joblib.load(classifier_info_name)
    
    results = {}
    
    for novelty_class, novelty_label in enumerate(class_labels):
        results[novelty_class] = {}
        
        # Class Specialist Efficiency
        for known_class, known_label in enumerate(class_labels):
            if known_class == novelty_class: continue
            results[novelty_class][known_label] = np.zeros(
                [len(train_info['nu_values']), 
                 len(train_info['CVO_novelty_%s'%(novelty_label)])])
        
        # Accuracy
        results[novelty_class]['Accuracy'] = np.zeros(
            [len(train_info['nu_values']), 
             len(train_info['CVO_novelty_%s'%(novelty_label)])]) 
        
        # Known SP
        results[novelty_class]['Known SP'] = np.zeros(
            [len(train_info['nu_values']), 
             len(train_info['CVO_novelty_%s'%(novelty_label)])])
        
        # Novelty Detection
        results[novelty_class]['Novelty Detection'] = np.zeros(
            [len(train_info['nu_values']), 
             len(train_info['CVO_novelty_%s'%(novelty_label)])])
        
        # Trigger
        results[novelty_class]['Trigger'] = np.zeros(
            [len(train_info['nu_values']), 
             len(train_info['CVO_novelty_%s'%(novelty_label)])])
       
        # for: folds
        for ifold in range(len(train_info['CVO_novelty_%s'%(novelty_label)])):
            # split data in trn set, tst set
            train_id, test_id = train_info['CVO_novelty_%s'%(novelty_label)][ifold]
            
            # normalize data based in train set
            if train_info['norm'] == 'mapstd':
                scaler = preprocessing.StandardScaler().fit(known_data[train_id,:])
            elif train_info['norm'] == 'mapstd_rob':
                scaler = preprocessing.RobustScaler().fit(known_data[train_id,:])
            elif train_info['norm'] == 'mapminmax':
                scaler = preprocessing.MinMaxScaler().fit(known_data[train_id,:])
        
            norm_all_data = scaler.transform(all_data)
        
            pca_norm_all_data = pcas[novelty_class][ifold].transform(norm_all_data)
            
            #cut some pcas
            pca_norm_all_data = pca_norm_all_data[:,0:train_info['n_pcas']]
            
            for nu_id, nu_value in enumerate(train_info['nu_values']):
                print 'Analysing Classifiers: Novelty Class: %s, %i of %i folds, %i of %i nu_values'%(
                    novelty_label, ifold+1, len(train_info['CVO_novelty_%s'%(novelty_label)]),
                    nu_id+1, len(train_info['nu_values'])
                )
                
                # class specialist
                for known_class, known_label in enumerate(class_labels):
                    if known_class == novelty_class: continue
                    output = (classifiers[novelty_class]
                              [ifold][nu_value]
                              [known_label].predict(pca_norm_all_data[all_trgt==known_class]))
                    
                    results[novelty_class][known_label][nu_id,ifold] = (
                    float(sum(output==1))/float(output.shape[0]))
                
                # accuracy
                buff = np.zeros([len(class_labels)-1,1])
                for known_class, known_label in enumerate(class_labels):
                    if known_class == novelty_class: continue
                    buff[known_class-(known_class>novelty_class)] = (
                        results[novelty_class][known_label][nu_id,ifold])
                results[novelty_class]['Accuracy'][nu_id,ifold] = np.mean(buff,axis=0)
                
                # known sp
                results[novelty_class]['Known SP'][nu_id,ifold] = (np.sqrt(np.mean(buff,axis=0)
                                                                           *sp_stats.gmean(buff,axis=0)))
                
                # novelty detection
                output = (classifiers[novelty_class]
                          [ifold][nu_value]
                          ['NoveltyDetector'].predict(pca_norm_all_data
                                                [all_trgt==novelty_class]))
                results[novelty_class]['Novelty Detection'][nu_id,ifold] = (
                float(sum(output==-1))/float(output.shape[0]))
                
                # trigger
                output = (classifiers[novelty_class]
                          [ifold][nu_value]
                          ['NoveltyDetector'].predict(pca_norm_all_data
                                                [all_trgt!=novelty_class]))
                results[novelty_class]['Trigger'][nu_id,ifold] = (
                float(sum(output==1))/float(output.shape[0]))
                
    result_file_path = result_analysis_path+'/result_files'+'/'+choose_date+'_results.jbl'
    joblib.dump([results],result_file_path,compress=9)

    train_info_name = result_analysis_path+'/train_info_files'+'/'+choose_date+'_train_info.jbl'

    train_info['results_done'] = True
    joblib.dump([train_info],train_info_name,compress=9)

In [ ]:
# Analise dos resultados
%time

import matplotlib.pyplot as plt

plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3

%matplotlib inline

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'PCASingleClassSVM':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'PCA analysis performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    
    # checking preprocessing
    if not train_info['preprocessing_done']:
        print 'Extract Preprocessing!!!'
        continue
        
    # checking training
    #if not train_info['train_done']:
    #    print 'Perform Train!!!'
    #    continue
        
    # checking results extraction
    if not train_info['results_done']:
        print 'Perform Extraction!!!'
        continue
        
    [pcas] = joblib.load(result_analysis_path+'/result_files'+'/'+choose_date+'_pca_file.jbl')
    
    #classifier_info_name = result_analysis_path+'/classifiers_files'+'/'+choose_date+'_classifiers.jbl'
    #[classifiers] = joblib.load(classifier_info_name)
    
    result_file_path = result_analysis_path+'/result_files'+'/'+choose_date+'_results.jbl'
    [results] = joblib.load(result_file_path)
    
    # Plot Efficiency
    fig, subplot_array = plt.subplots(nrows=2, ncols=2,figsize=(20,20))
    m_colors = ['b', 'r', 'g', 'y']
    for novelty_class, novelty_label in enumerate(class_labels):
        ax = plt.subplot(2,2,novelty_class+1)
        m_fontsize = 18
        plt.title('Classifier Eff. - Novelty: '+novelty_label, fontsize= m_fontsize, fontweight="bold")
        if novelty_class > -1:
            plt.xlabel(r'$\nu$ values', fontsize= m_fontsize, fontweight="bold")
        plt.ylabel('Efficiency(%)', fontsize= m_fontsize, fontweight="bold")
        m_leg = []
        
        line_width = 3.5
        
        # class specialist eff
        for known_class, known_label in enumerate(class_labels):
            if known_class == novelty_class: continue
            plot_data = results[novelty_class][known_label]
            ax.errorbar(train_info['nu_values'],
                        100*np.mean(plot_data,axis=1),
                        100*np.std(plot_data,axis=1),marker='o',
                        color=m_colors[known_class],alpha=0.5,linewidth=line_width)
            m_leg.append(known_label+' Eff.')
        
        # accuracy
        plot_data = results[novelty_class]['Accuracy']
        ax.errorbar(train_info['nu_values'],
                    100*np.mean(plot_data,axis=1),
                    100*np.std(plot_data,axis=1),marker='o',
                    color='k',ls='-.',alpha=0.5, linewidth=line_width)
        m_leg.append('Known Acc.')
        
        # known sp
        plot_data = results[novelty_class]['Known SP']
        ax.errorbar(train_info['nu_values'],
                    100*np.mean(plot_data,axis=1),
                    100*np.std(plot_data,axis=1),marker='o',
                    color='k',ls='-.',alpha=0.5, linewidth=line_width)
        m_leg.append('Known SP')
        
        # trigger
        plot_data = results[novelty_class]['Trigger']
        ax.errorbar(train_info['nu_values'],
                    100*np.mean(plot_data,axis=1),
                    100*np.std(plot_data,axis=1),marker='o',
                    color='k',ls=':',alpha=0.5, linewidth=line_width)
        m_leg.append('Trigger')
        
        # novelty detection
        plot_data = results[novelty_class]['Novelty Detection']
        ax.errorbar(train_info['nu_values'],
                    100*np.mean(plot_data,axis=1),
                    100*np.std(plot_data,axis=1),marker='o',
                    color='k',ls='-',alpha=0.5, linewidth=line_width)
        m_leg.append('Novelty Detection')
        
        # graphical assusts
        ax.set_ylim([0.0, 115])
        ax.set_yticks([x for x in range(0,101,5)])
        
        ax.set_xlim([np.min(train_info['nu_values']), np.max(train_info['nu_values'])])
        ax.set_xticks(train_info['nu_values'])
        ax.set_xticklabels(train_info['nu_values'],rotation=45)
        
        ax.grid()
        ax.legend(m_leg, loc='upper right',ncol=3)
    fig.savefig(result_analysis_path+'/picts/'+choose_date+'_'+
                log_entries[log_id]['package']+'_novelty_detection.pdf')

In [ ]:
print result_analysis_path+'/picts/'+choose_date+'_'+log_entries[log_id]['package']+'_novelty_detection.pdf'
